In [1]:
import pandas as pd 

In [2]:
dataset=pd.read_csv('insurance_pre.csv')

In [3]:
dataset.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges'], dtype='object')

In [4]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [5]:
dataset=pd.get_dummies(dataset)

In [6]:
dataset

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,16884.92400,1,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0
2,28,33.000,3,4449.46200,0,1,1,0
3,33,22.705,0,21984.47061,0,1,1,0
4,32,28.880,0,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0
1334,18,31.920,0,2205.98080,1,0,1,0
1335,18,36.850,0,1629.83350,1,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0


In [7]:
independent=dataset[['age', 'bmi', 'children', 'sex_female', 'sex_male','smoker_no', 'smoker_yes']]
independent

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,1,0,0,1
1,18,33.770,1,0,1,1,0
2,28,33.000,3,0,1,1,0
3,33,22.705,0,0,1,1,0
4,32,28.880,0,0,1,1,0
...,...,...,...,...,...,...,...
1333,50,30.970,3,0,1,1,0
1334,18,31.920,0,1,0,1,0
1335,18,36.850,0,1,0,1,0
1336,21,25.800,0,1,0,1,0


In [8]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test=train_test_split(independent, dependent,test_size=0.30,random_state=0)

In [14]:
! pip install lightgbm
from sklearn.model_selection import GridSearchCV
import lightgbm as ltb
param_grid={'boosting_type':['gbdt','dart'],'n_estimators':[100,200,300],'max_depth':[15,30,45,60],'num_leaves':[10,20,30,40],'learning_rate':[0.1],'random_state':[None]}
grid= GridSearchCV(ltb.LGBMRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)   
grid.fit(independent,dependent) 


Fitting 5 folds for each of 96 candidates, totalling 480 fits


C:\Users\Jayarajan\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Jayarajan\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 245, in _count_physical_cores
    raise ValueError(


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 317
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 7
[LightGBM] [Info] Start training from score 13270.422260


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt', 'dart'],
                         'learning_rate': [0.1], 'max_depth': [15, 30, 45, 60],
                         'n_estimators': [100, 200, 300],
                         'num_leaves': [10, 20, 30, 40],
                         'random_state': [None]},
             verbose=3)

In [15]:
re=grid.cv_results_

In [16]:
print("The R_score value for the best parameter{}:".format(grid.best_params_))

The R_score value for the best parameter{'boosting_type': 'dart', 'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 300, 'num_leaves': 10, 'random_state': None}:


In [17]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_boosting_type,param_learning_rate,param_max_depth,param_n_estimators,param_num_leaves,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.639833,0.067831,0.008995,0.001548,gbdt,0.1,15,100,10,None,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.875109,0.800946,0.886444,0.847060,0.857800,0.853472,0.029577,9
1,0.960048,0.022953,0.007796,0.001166,gbdt,0.1,15,100,20,None,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.864817,0.790269,0.876511,0.838785,0.849994,0.844075,0.029801,29
2,1.358622,0.112239,0.008393,0.000801,gbdt,0.1,15,100,30,None,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.861190,0.779988,0.871975,0.836094,0.848030,0.839455,0.032097,49
3,1.744000,0.057170,0.009594,0.001018,gbdt,0.1,15,100,40,None,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.855962,0.779936,0.872633,0.831701,0.846240,0.837295,0.031618,57
4,1.026411,0.116514,0.006996,0.000631,gbdt,0.1,15,200,10,None,"{'boosting_type': 'gbdt', 'learning_rate': 0.1...",0.868540,0.791114,0.877003,0.842941,0.851969,0.846313,0.030087,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,3.758242,0.153534,0.008596,0.001356,dart,0.1,60,200,40,None,"{'boosting_type': 'dart', 'learning_rate': 0.1...",0.864150,0.787656,0.879556,0.831139,0.847760,0.842052,0.031640,37
92,2.467383,0.025957,0.006797,0.000400,dart,0.1,60,300,10,None,"{'boosting_type': 'dart', 'learning_rate': 0.1...",0.877835,0.805891,0.889014,0.845666,0.854060,0.854493,0.028903,1
93,3.732457,0.156882,0.008196,0.000979,dart,0.1,60,300,20,None,"{'boosting_type': 'dart', 'learning_rate': 0.1...",0.867710,0.794236,0.881171,0.840636,0.849696,0.846690,0.029759,17
94,4.733884,0.314684,0.009593,0.001493,dart,0.1,60,300,30,None,"{'boosting_type': 'dart', 'learning_rate': 0.1...",0.863401,0.786962,0.879564,0.835392,0.849287,0.842921,0.031594,34


In [18]:
age_input=int(input("Enter age: "))
bmi_input=float(input("Enter bmi: "))
children_input=float(input("Enter children: "))
gender_male_input=int(input("Enter Gender: "))
gender_female_input=int(input("Enter Gender: "))
smoker_yes_input=int(input("Enter smoker: "))
smoker_no_input=int(input("Enter smoker: "))

Enter age: 21
Enter bmi: 34.5
Enter children: 2
Enter Gender: 1
Enter Gender: 0
Enter smoker: 1
Enter smoker: 0


In [19]:
future_predict=grid.predict([[age_input,bmi_input,children_input,gender_male_input,gender_female_input,smoker_yes_input,smoker_no_input]])
print("Future Prediction{}".format(future_predict))

Future Prediction[5972.77174416]
